In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Spracovanie textu

URL https://github.com/FIIT-IAU


## Extrakcia čŕt z textu

Aby sme mohli text klasifikovať, hľadať zhluky podobných dokumentov a pod.

#### Príklad: chceme rozlíšiť, kto je autorom textu

Edgar Allan Poe vs. Mary Shelley vs. HP Lovecraft: https://www.kaggle.com/c/spooky-author-identification

**Aké črty (features) by som mohol extrahovať z viet?**
* Dĺžka vety
* Počet slov vo vete
* Priemerná dĺžka slov vo vete
* Zložitosť vety (tzv. text readability metrics, napr. Flesh-Kincaid)
* Počet spojok/predložiek/iných slovných druhov
* **Frekvencia použitých slov - prevod vety (textu) do vektorovej reprezentácie**

#### Vo všeobecnosti

* Segmentácia textu 
* Prevod textu do vektorovej reprezentácie (tzv. *bag of words*)
* Identifikácia kľúčových slov, resp. často sa spolu vyskytujúcich slov (tokenov)
* Určovanie podobnosti dvoch textových dokumentov

## Metódy na spracovanie textu
- Regulárne výrazy, konečné automaty, bezkontextové gramatiky
- Pravidlové, slovníkové prístupy
- Prístupy strojového učenia (Markovovské modely, **hlboké neurónové siete**)


#### Väčšina metód je jazykovo-závislá
- Veľa dostupných modelov pre angličtinu, nemčinu, španielčinu, ...


#### Reprezentácia textu
- Textový dokument väčšinou reprezentujeme pomocou množiny slov (angl. *bag-of-words*) = **vektorom**.
- Zložky vektora predstavujú jednotlivé slová, resp. n-gramy zo slovníka (pre celý korpus/jazyk).

Hodnotou zložiek vektora môže byť:
* výskyt (binárne)
* početnosť
* frekvencia
* váhovaná frekvencia

#### Prevod textu na vektor

1. Tokenizácia (rozdelenie textu na vety, následne na slová)
2. Normalizácia textu
   - prevod na malé písmená
   - stemming alebo lematizácia
   - odstránenie stopslov (spojky, predložky a pod.)
3. Vytvorenie slovníka
4. Vytvorenie vektora - zložky slová zo slovníka; väčšinou riedky (angl. *sparse*; veľa núl)

## Tokenizácia

In [ ]:
import nltk
# nltk.download('punkt')

text = """At eight o'clock on Thursday morning 
... Arthur didn't feel very good. He closed his eyes and went to bed again."""

In [ ]:
sentences = nltk.sent_tokenize(text)
print(sentences)

In [ ]:
sent = sentences[0]

tokens = nltk.word_tokenize(sent)
print(tokens)

## Normalizácia

In [ ]:
tokens = [token.lower() for token in tokens if token not in ".,?!..."]
print(tokens)

## Stemming alebo lematizácia?

- Stemming vráti koreň slova. Napr. *ryba -> ryb*
- Lematizácia prevádza slová na ich základný slovníkový tvar. Napr. *rybe -> ryba*
- **Je to vždy jedno alebo druhé.** Prevod na koreň slova sa používa viac pri málo flexných jazykoch (napr. angličtina). Pri flexných jazykoch (napr. slovenčina) je preferovaná lematizácia.
- **Stemming** - pre angličtinu napr. [Porterov algoritmus (1980)](https://www.cs.odu.edu/~jbollen/IR04/readings/readings5.pdf)
- **Lematizácia** - väčšinou slovníkové metódy (morfologická databáza, resp. tvaroslovník); pre slovenčinu: https://korpus.sk/morphology_database.html

In [ ]:
porter = nltk.PorterStemmer()

stemmed = [porter.stem(token) for token in tokens]
print(stemmed)

In [ ]:
# nltk.download('wordnet')

wnl = nltk.WordNetLemmatizer()

lemmatized = [wnl.lemmatize(token) for token in tokens]
print(lemmatized)

### Odstránenie stopslov

In [ ]:
# nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')

normalized_tokens = [token for token in stemmed if token not in stopwords]
print(normalized_tokens)

## Prevod na vektorovú reprezentáciu

Datasetom [20 newsgroups](http://qwone.com/~jason/20Newsgroups/):

*"The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. To the best of my knowledge, it was originally collected by Ken Lang, probably for his Newsweeder: Learning to filter netnews paper, though he does not explicitly mention this collection. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering."*

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [ ]:
twenty_train.target_names

In [ ]:
len(twenty_train.data)

In [ ]:
print("\n".join(twenty_train.data[0].split("\n")[:10]))

In [ ]:
def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')
    return [token.lower() for token in tokens if token.isalpha() and token.lower() not in stopwords]

In [ ]:
tokenized_docs = [preprocess_text(text) for text in twenty_train.data]

In [ ]:
print(tokenized_docs[0])

In [ ]:
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]
print(corpus[10])

## TF-IDF = term frequency * inverse document frequency

`TF` – frekvencia slova v aktuálnom dokumente

`IDF` – záporný logaritmus pravdepodobnosti výskytu slova v dokumentoch korpusu (rovnaká pre všetky dokumenty)

### $ tf(t,d)=\frac{f_{t,d}}{\sum_{t' \in d}{f_{t',d}}} $

### $ idf(t,D) = -\log_2{\frac{|{d \in D: t \in d}|}{N}} = \log_2{\frac{N}{|{d \in D: t \in d}|}} $

Rôzne varianty (váhovacie schémy): https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [ ]:
tfidf_model = models.TfidfModel(corpus)
tfidf_corpus = tfidf_model[corpus]
tfidf_corpus[10][:10]

## Podobnosť vektorov

Podobnosť pomocou Euklidovskej vzdialenosti

### $ sim(u,v) = 1- d(u,v) = 1 - \sqrt{\sum_{i=1}^{n}{(v_i-u_i)^2}} $

Kosínusová podobnosť

### $sim(u,v) = cos(u,v) = \frac{u \cdot v}{||u||||v||} =\frac{\sum_{i=1}^{n}{u_iv_i}}{\sum_{i=1}^{n}{u_i}\sum_{i=1}^{n}{v_i}} $

In [ ]:
index = similarities.MatrixSimilarity(tfidf_corpus)
index[tfidf_corpus[0]]

## Extrakcia čŕt pomocou scikit-learn

http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

In [ ]:
print(count_vect.vocabulary_.get(u'algorithm'))

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

Natrénujeme klasifikátor

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [ ]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

## Sprehľadnenie a automatizácia predspracovania: Pipelines

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

In [ ]:
from sklearn.pipeline import Pipeline

text_ppl = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())
                    ])

In [ ]:
text_ppl.fit(twenty_train.data, twenty_train.target)

In [ ]:
[twenty_train.target_names[cat] for cat in text_ppl.predict(docs_new)]

## Vlastný transformátor

In [ ]:
from sklearn.base import TransformerMixin

class MyTransformer(TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        return X

# Iné časté úlohy (pred)spracovania textu

## Rozpoznávanie slovných druhov = Part-of-Speech Tagging (POS)

Slovný druh, číslo, čas, prípadne ďalšie gramatické kategórie

In [ ]:
# nltk.download('averaged_perceptron_tagger')

tagged = nltk.pos_tag(nltk.word_tokenize(sent))
print(tagged)

In [ ]:
# nltk.download('tagsets')

nltk.help.upenn_tagset('NNP')

## Rozpoznávanie menných entít = Name Entity Recognition (NER)

Osoby, organizácie, miesta a pod.

In [ ]:
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

entities = nltk.chunk.ne_chunk(tagged)

In [ ]:
print(entities.__repr__())

## N-gramy

Vo všobecnosti ide o postupnosť $N$ po sebe idúcich položiek. V texte väčšinou na úrovni slov.
- bigramy
- trigramy
- skipgramy - $k$-skip-$n$-gramy
- https://books.google.com/ngrams

In [ ]:
tokens = nltk.word_tokenize(sent)
bigrams = list(nltk.bigrams(tokens))
print(bigrams[:5])

Dá sa nastaviť aj v `CountVectorizer` transformátore.

In [ ]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
analyze = bigram_vectorizer.build_analyzer()
analyze('Bi-grams are cool!')

## WordNet

* Lexikálna databáza
* Organizovaná pomocou tzv. synsetov (množín synoným)
  * Podstatné mená, slovesá, prídavné mená, príslovky
* Prepojenia medzi synsetmi
  * Antonymá, hyperonymá, hyponymá, holonymá, meronymá

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
print(wn.synsets('car'))

In [ ]:
car = wn.synset('car.n.01')

In [ ]:
car.lemma_names()

In [ ]:
car.definition()

In [ ]:
car.examples()

In [ ]:
print(car.hyponyms()[:5])

In [ ]:
car.hypernyms()

In [ ]:
print(car.part_meronyms()[:5])

In [ ]:
wn.synsets('black')[0].lemmas()[0].antonyms()

## Vektorová reprezentácia slov - word2vec

Každé slovo má naučený vektor reálnych čísel, ktoré reprezentujú rôzne jeho vlastnosti a zachytávajú viaceré lingvistické pravidelnosti. Môžeme počítať podobnosť medzi slovami ako podobnosť dvoch vektorov.

vector('Paris') - vector('France') + vector('Italy') ~= vector('Rome')

vector('king') - vector('man') + vector('woman') ~= vector('queen')

- https://radimrehurek.com/gensim/models/word2vec.html
- https://medium.com/@mishra.thedeepak/word2vec-in-minutes-gensim-nlp-python-6940f4e00980

In [ ]:
from nltk.corpus import brown
nltk.download('brown')

sentences = brown.sents()
model = models.Word2Vec(sentences, min_count=1)
model.save('brown_model')
model = models.Word2Vec.load('brown_model')

In [ ]:
# print(model.most_similar("mother"))
print(model.wv.most_similar("mother"))

In [ ]:
# print(model.doesnt_match("breakfast cereal dinner lunch".split()))
print(model.wv.doesnt_match("breakfast cereal dinner lunch".split()))

# Užitočné slovníky
- ConceptNet: http://conceptnet.io/
- Sentiment a emócie: [WordNet-Affect](http://wndomains.fbk.eu/wnaffect.html), 
- [SenticNet](https://sentic.net/), 
- [EmoSenticNet](https://www.gelbukh.com/emosenticnet/)


# Nástroje na spracovanie textu v Pythone

- [NLTK](http://www.nltk.org/)
- [Gensim](https://radimrehurek.com/gensim/tutorial.html)
- [sklearn.feature_extraction.text](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

#### Nástroje (mimo Pythonu)
- [Stanford CoreNLP](https://stanfordnlp.github.io/CoreNLP/) - rozhranie aj cez NLTK
- [Apache OpenNLP](https://opennlp.apache.org/)
- [WordNet](https://wordnet.princeton.edu/) - rozhranie cez NLTK


# Extrakcia čŕt sa robí aj s inými typmi vstupov
- Obrázky (sklearn.feature_extraction.image, [scikit-image](https://scikit-image.org/))
- Videá ([scikit-video](http://www.scikit-video.org/stable/))
- Signál, napr. zvuk ([scikit-signal](https://docs.scipy.org/doc/scipy/reference/signal.html), [scikit-sound](http://work.thaslwanter.at/sksound/html/))


# Ďalšie lingvistické modely
- [fastText](https://fasttext.cc/), [ELMo](https://allennlp.org/elmo), [BERT](https://github.com/google-research/bert), [GloVe](https://nlp.stanford.edu/projects/glove/): nejaké základné porovnanie [tu](https://www.quora.com/What-are-the-main-differences-between-the-word-embeddings-of-ELMo-BERT-Word2vec-and-GloVe)
- [sentence embeddings](https://github.com/oborchers/Fast_Sentence_Embeddings)
- [doc2vec](https://radimrehurek.com/gensim/models/doc2vec.html)
- ...a ďalšie

# Pre slovenčinu

- [NLP4SK](http://arl6.library.sk/nlp4sk/)
- [Slovenský národný korpus](https://korpus.sk/)
- [word2vec](https://github.com/essential-data/word2vec-sk)
- a [ďalšie...](https://github.com/essential-data/nlp-sk-interesting-links)


# Zdroje
- [Dan Jurafsky, James H. Martin: Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/)
- http://www.nltk.org/book/
- https://radimrehurek.com/gensim/tutorial.html
- https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html